In [ ]:
import os

# --- 1. CẤU HÌNH GỐC (ROOT) ---
# Dùng r'' để Python hiểu đây là đường dẫn Windows (không bị lỗi ký tự đặc biệt)
ROOT = r'E:\CAFA-6-Protein-Function-Prediction'

# --- 2. ĐỊNH NGHĨA CÁC THƯ MỤC CON (Tự động nối đuôi) ---
# os.path.join giúp nối đường dẫn đúng chuẩn cho cả Windows/Linux/Mac
INPUT_DIR  = os.path.join(ROOT, 'input')
OUTPUT_DIR = os.path.join(ROOT, 'output')
MODEL_DIR  = os.path.join(ROOT, 'models')

# Tạo sẵn thư mục output và models nếu chưa có (Tránh lỗi không lưu được file)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"✅ Đã cấu hình ROOT: {ROOT}")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"📂 Models: {MODEL_DIR}")

✅ Đã cấu hình ROOT: E:\CAFA-6-Protein-Function-Prediction
📂 Input: E:\CAFA-6-Protein-Function-Prediction\input
📂 Output: E:\CAFA-6-Protein-Function-Prediction\output
📂 Models: E:\CAFA-6-Protein-Function-Prediction\models


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
import os
import gc
import obonet
import networkx as nx
import random # Needed for seeding

# ==========================================
# 0. REPRODUCIBILITY (SEED EVERYTHING)
# ==========================================
def seed_everything(seed=42):
    """
    Sets the seed for generating random numbers to ensure reproducibility.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # For multi-GPU
    # Deterministic algorithms (makes it slower but reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Locked Random Seed: {seed}")

# Apply seed immediately
seed_everything(42)

# ==========================================
# 1. CONFIGURATION PATHS (MATCHING YOUR IMAGE)
# ==========================================
class Config:
    # Root Dir (Main dataset folder on Kaggle)
    # Note: Folder name might vary depending on when data was added, check right panel
    RAW_DATA_DIR = f'{INPUT_DIR}/cafa-6-protein-function-prediction'
    EMBED_DIR = f'{INPUT_DIR}/cafa-6-t5-embeddings'
    
    # Sub-folder structure (Based on image 2)
    TRAIN_DIR = os.path.join(RAW_DATA_DIR, 'Train')
    TEST_DIR = os.path.join(RAW_DATA_DIR, 'Test')
    
    # File Paths
    TRAIN_TERMS = os.path.join(TRAIN_DIR, 'train_terms.tsv')
    TRAIN_TAXONOMY = os.path.join(TRAIN_DIR, 'train_taxonomy.tsv')
    OBO_FILE = os.path.join(TRAIN_DIR, 'go-basic.obo')
    
    # Test Taxonomy (Crucial for boosting Test set score)
    TEST_TAXONOMY = os.path.join(TEST_DIR, 'testsuperset-taxon-list.tsv')
    
    # Embeddings Paths (Based on image 1)
    TRAIN_EMBEDS = os.path.join(EMBED_DIR, 'train_embeds.npy')
    TRAIN_IDS = os.path.join(EMBED_DIR, 'train_ids.npy')
    TEST_EMBEDS = os.path.join(EMBED_DIR, 'test_embeds.npy')
    TEST_IDS = os.path.join(EMBED_DIR, 'test_ids.npy')
    
    # Hyperparameters
    NUM_CLASSES = 1500      # Number of GO Terms (Labels)
    BATCH_SIZE = 256        # Large batch for faster GPU run
    EPOCHS = 12             # Epochs per fold
    LR = 1e-3
    N_FOLDS = 5             # Ensemble 5 models
    TOP_TAXONS_COUNT = 50   # Top 50 most common species for one-hot
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = Config()
print(f"Running on: {cfg.DEVICE}")
print(f"Train Taxonomy Path: {cfg.TRAIN_TAXONOMY}")

# ==========================================
# 2. DATA PROCESSING & TAXONOMY
# ==========================================
# ==========================================
# UPDATE: ID PROCESSING FUNCTION TO FIX N_SAMPLES=0 ERROR
# ==========================================

def clean_ids(id_list):
    """
    Converts IDs from 'sp|P12345|XYZ' to 'P12345' format if needed.
    """
    cleaned = []
    for x in id_list:
        # Convert bytes to string if needed
        if isinstance(x, bytes):
            x = x.decode('utf-8')
        x = str(x).strip()
        
        # Handle sp|Accession|Name format
        if '|' in x:
            # Extract middle part (Accession ID)
            parts = x.split('|')
            if len(parts) >= 2:
                cleaned.append(parts[1]) # Gets P12345
            else:
                cleaned.append(x)
        else:
            cleaned.append(x)
    return np.array(cleaned)

def load_and_align_data_fixed():
    print("--- 1. Loading Embeddings & Fixing IDs ---")
    X_train = np.load(cfg.TRAIN_EMBEDS)
    train_ids_raw = np.load(cfg.TRAIN_IDS)
    
    X_test = np.load(cfg.TEST_EMBEDS)
    test_ids_raw = np.load(cfg.TEST_IDS)
    
    # --- CRITICAL FIX STEP: CLEAN IDS ---
    train_ids = clean_ids(train_ids_raw)
    test_ids = clean_ids(test_ids_raw)
    
    # DEBUG: Print to check if IDs match
    print(f"Sample Raw Train ID: {train_ids_raw[0]}")
    print(f"Sample Cleaned Train ID: {train_ids[0]}")
    print(f"Train shape: {X_train.shape}")
    
    # --- PROCESS TAXONOMY ---
    print("--- 2. Processing Taxonomy Features ---")
    
    # Read Taxonomy file
    train_tax_df = pd.read_csv(cfg.TRAIN_TAXONOMY, sep='\t', dtype=str)
    # Map: ProteinID -> TaxonID
    # Note: First column of taxonomy must also match cleaned ID
    train_tax_map = dict(zip(train_tax_df.iloc[:, 0], train_tax_df.iloc[:, 1]))
    
    test_tax_df = pd.read_csv(cfg.TEST_TAXONOMY, sep='\t', dtype=str)
    test_tax_map = dict(zip(test_tax_df.iloc[:, 0], test_tax_df.iloc[:, 1]))
    
    top_taxons = train_tax_df.iloc[:, 1].value_counts().head(cfg.TOP_TAXONS_COUNT).index.tolist()
    tax2idx = {t: i for i, t in enumerate(top_taxons)}
    
    def create_tax_features(ids, tax_map):
        feats = np.zeros((len(ids), cfg.TOP_TAXONS_COUNT), dtype=np.float32)
        match_count = 0
        for i, pid in enumerate(ids):
            if pid in tax_map:
                match_count += 1
                tid = tax_map[pid]
                if tid in tax2idx:
                    feats[i, tax2idx[tid]] = 1.0
        print(f"   > Found taxonomy for {match_count}/{len(ids)} proteins")
        return feats

    print("Generating Taxon Vectors...")
    train_tax_feats = create_tax_features(train_ids, train_tax_map)
    test_tax_feats = create_tax_features(test_ids, test_tax_map)
    
    X_train_final = np.concatenate([X_train, train_tax_feats], axis=1)
    X_test_final = np.concatenate([X_test, test_tax_feats], axis=1)
    
    return X_train_final, train_ids, X_test_final, test_ids

# --- HÀM TÍNH F-MAX ---
def calculate_fmax(preds, targets):
    thresholds = np.linspace(0.01, 1.0, 50)
    fmax = 0.0
    best_t = 0.0
    targets = targets.astype(int)
    
    for t in thresholds:
        p_cut = (preds >= t).astype(int)
        tp = (p_cut * targets).sum(axis=1)
        fp = (p_cut * (1 - targets)).sum(axis=1)
        fn = ((1 - p_cut) * targets).sum(axis=1)
        
        precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp, dtype=float), where=(tp + fp) != 0)
        recall = np.divide(tp, (tp + fn), out=np.zeros_like(tp, dtype=float), where=(tp + fn) != 0)
        
        p_avg = precision.mean()
        r_avg = recall.mean()
        
        if (p_avg + r_avg) > 0:
            f1 = 2 * p_avg * r_avg / (p_avg + r_avg)
        else:
            f1 = 0
            
        if f1 > fmax:
            fmax = f1
            best_t = t
            
    return fmax, best_t
# ==========================================
# 3. MODEL: RESNET (Deep & Residual)
# ==========================================
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim)
        )
        self.relu = nn.ReLU()
    def forward(self, x):
        return self.relu(x + self.net(x)) # Skip Connection

class AdvancedModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Input Projection: Project vector (1074) to higher dimension
        self.entry = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        # Deep Residual Layers
        self.blocks = nn.Sequential(
            ResidualBlock(1024),
            ResidualBlock(1024),
            ResidualBlock(1024)
        )
        self.head = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.entry(x)
        x = self.blocks(x)
        return self.head(x)

# ==========================================
# 4. TRAINING UTILS
# ==========================================
class ProteinDataset(Dataset):
    def __init__(self, X, Y=None):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float() if Y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): 
        return (self.X[i], self.Y[i]) if self.Y is not None else self.X[i]

def train_one_fold(fold, model, train_loader, val_loader):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=cfg.LR, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)
    
    best_loss = float('inf')
    early_stop_count = 0
    patience = 3 
    
    print(f"--- FOLD {fold+1} STARTED ---")
    
    for epoch in range(cfg.EPOCHS):
        model.train()
        t_loss = 0
        # Train loop
        for x, y in train_loader:
            x, y = x.to(cfg.DEVICE), y.to(cfg.DEVICE)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            t_loss += loss.item()
            
        # Validation loop (CẬP NHẬT ĐỂ TÍNH F-MAX)
        model.eval()
        v_loss = 0
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(cfg.DEVICE), y.to(cfg.DEVICE)
                logits = model(x)
                v_loss += criterion(logits, y).item()
                
                # Lưu lại dự đoán để tính F-max
                # Phải dùng Sigmoid vì output của model là Logits
                val_preds.append(torch.sigmoid(logits).cpu().numpy())
                val_targets.append(y.cpu().numpy())
        
        avg_v_loss = v_loss / len(val_loader)
        
        # Gộp các batch lại và tính F-max
        val_preds = np.concatenate(val_preds)
        val_targets = np.concatenate(val_targets)
        fmax_score, best_t = calculate_fmax(val_preds, val_targets)
        
        scheduler.step(avg_v_loss)
        
        # In kết quả ra màn hình
        print(f"Ep {epoch+1}: Loss={avg_v_loss:.4f} | F-max={fmax_score:.4f} (threshold={best_t:.2f})")
        
        # Save Best Model
        if avg_v_loss < best_loss:
            best_loss = avg_v_loss
            torch.save(model.state_dict(), f'model_fold_{fold}.pth')
            early_stop_count = 0
        else:
            early_stop_count += 1
            
        if early_stop_count >= patience:
            print(f"  >> Fold {fold+1}: Early stopping at epoch {epoch+1}")
            break
    
    return best_loss

# ==========================================
# 5. MAIN PIPELINE (ENSEMBLE 5-FOLD)
# ==========================================

# --- LOAD DATA WITH NEW FUNCTION ---
X_train_full, train_ids_full, X_test_full, test_ids_full = load_and_align_data_fixed()

print("--- 3. Processing Labels & Checking Mismatch ---")
df = pd.read_csv(cfg.TRAIN_TERMS, sep='\t', names=['EntryID', 'term', 'aspect'])
print(f"Sample ID in Train Terms: {df['EntryID'].iloc[0]}") # Debug to check ID format in labels file

# Get Top Labels
top_terms = df['term'].value_counts().head(cfg.NUM_CLASSES).index.tolist()
term2idx = {t: i for i, t in enumerate(top_terms)}

# Map ID -> Set of Terms
id_to_terms = df[df['term'].isin(top_terms)].groupby('EntryID')['term'].apply(set).to_dict()

# Create Matrix Y
Y_full = np.zeros((len(train_ids_full), cfg.NUM_CLASSES), dtype=np.float32)
valid_mask = []

for i, pid in enumerate(train_ids_full):
    if pid in id_to_terms:
        valid_mask.append(True)
        for term in id_to_terms[pid]:
            Y_full[i, term2idx[term]] = 1.0
    else:
        valid_mask.append(False)

valid_mask = np.array(valid_mask)
print(f"DEBUG: Total proteins: {len(train_ids_full)}")
print(f"DEBUG: Matched proteins with labels: {sum(valid_mask)}")

if sum(valid_mask) == 0:
    raise ValueError("CRITICAL ERROR: No IDs matched! Check the 'Sample Cleaned Train ID' vs 'Sample ID in Train Terms' printed above.")

# Keep only matched data
X_train_clean = X_train_full[valid_mask]
Y_clean = Y_full[valid_mask]

# --- CONTINUE TRAINING LOOP ---
kf = KFold(n_splits=cfg.N_FOLDS, shuffle=True, random_state=42)
input_dim = X_train_clean.shape[1]

print(f"\n--- Starting Ensemble Training ({cfg.N_FOLDS} Folds) ---")
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_clean)):
    print(f">> Fold {fold+1}/{cfg.N_FOLDS}")
    
    train_ds = ProteinDataset(X_train_clean[train_idx], Y_clean[train_idx])
    val_ds = ProteinDataset(X_train_clean[val_idx], Y_clean[val_idx])
    
    train_loader = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=cfg.BATCH_SIZE)
    
    model = AdvancedModel(input_dim, cfg.NUM_CLASSES).to(cfg.DEVICE)
    train_one_fold(fold, model, train_loader, val_loader)
    
    del model, train_loader, val_loader, train_ds, val_ds
    torch.cuda.empty_cache()
    gc.collect()


# C. INFERENCE & AVERAGING
print("\n--- Running Inference on Test Set ---")
test_loader = DataLoader(ProteinDataset(X_test_full), batch_size=cfg.BATCH_SIZE)
final_preds = np.zeros((len(test_ids_full), cfg.NUM_CLASSES), dtype=np.float32)

for fold in range(cfg.N_FOLDS):
    model = AdvancedModel(input_dim, cfg.NUM_CLASSES).to(cfg.DEVICE)
    model.load_state_dict(torch.load(f'model_fold_{fold}.pth'))
    model.eval()
    
    fold_preds = []
    with torch.no_grad():
        for x in tqdm(test_loader, desc=f"Predicting Fold {fold+1}"):
            x = x.to(cfg.DEVICE)
            logits = model(x)
            fold_preds.append(torch.sigmoid(logits).cpu().numpy())
            
    final_preds += np.vstack(fold_preds)

final_preds /= cfg.N_FOLDS # Average results of 5 folds

# D. GO PROPAGATION (SCORE HACK)
print("--- Applying GO Structure Propagation ---")
# Load GO Graph
go_graph = obonet.read_obo(cfg.OBO_FILE)
term_to_idx = {t: i for i, t in enumerate(top_terms)}

# Find parents for each term
parent_map = {}
valid_set = set(top_terms)
for term in top_terms:
    if term in go_graph:
        parents = list(go_graph.successors(term))
        for p in parents:
            if p in valid_set:
                if p not in parent_map: parent_map[p] = []
                parent_map[p].append(term)

# Propagate: Score Parent = Max(Score Parent, Max(Score Child))
# Run 2 passes to ensure propagation up the hierarchy
for _ in range(2):
    for parent, children in parent_map.items():
        if children:
            p_idx = term_to_idx[parent]
            c_indices = [term_to_idx[c] for c in children]
            max_child_score = final_preds[:, c_indices].max(axis=1)
            final_preds[:, p_idx] = np.maximum(final_preds[:, p_idx], max_child_score)

# E. SUBMISSION FILE
print("--- Creating Submission File ---")
submission_rows = []
threshold = 0.005 # Low threshold to keep more results (Kaggle metric calculates weighted F1)

for i, pid in enumerate(tqdm(test_ids_full)):
    # Use numpy where for fast filtering
    idxs = np.where(final_preds[i] > threshold)[0]
    for idx in idxs:
        term = top_terms[idx]
        score = final_preds[i, idx]
        submission_rows.append(f"{pid}\t{term}\t{score:.3f}")

with open(f'{OUTPUT_DIR}submission_dl.tsv', 'w') as f:
    # f.write("ProteinID\tGO_ID\tScore\n") # Uncomment if header is required
    f.write('\n'.join(submission_rows))

print("DONE! File submission_dl.tsv is ready in Output tab.")

Locked Random Seed: 42
Running on: cuda
Train Taxonomy Path: /kaggle/input/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv
--- 1. Loading Embeddings & Fixing IDs ---
Sample Raw Train ID: sp|A0A0C5B5G6|MOTSC_HUMAN
Sample Cleaned Train ID: A0A0C5B5G6
Train shape: (82404, 1024)
--- 2. Processing Taxonomy Features ---
Generating Taxon Vectors...
   > Found taxonomy for 82403/82404 proteins
   > Found taxonomy for 0/224309 proteins
--- 3. Processing Labels & Checking Mismatch ---
Sample ID in Train Terms: EntryID
DEBUG: Total proteins: 82404
DEBUG: Matched proteins with labels: 76308

--- Starting Ensemble Training (5 Folds) ---
>> Fold 1/5
>> Fold 2/5
>> Fold 3/5
>> Fold 4/5
>> Fold 5/5

--- Running Inference on Test Set ---


Predicting Fold 1:   0%|          | 0/877 [00:00<?, ?it/s]

Predicting Fold 2:   0%|          | 0/877 [00:00<?, ?it/s]

Predicting Fold 3:   0%|          | 0/877 [00:00<?, ?it/s]

Predicting Fold 4:   0%|          | 0/877 [00:00<?, ?it/s]

Predicting Fold 5:   0%|          | 0/877 [00:00<?, ?it/s]

--- Applying GO Structure Propagation ---
--- Creating Submission File ---


  0%|          | 0/224309 [00:00<?, ?it/s]

DONE! File submission.tsv is ready in Output tab.
